In [1]:
%config Completer.use_jedi = False

%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys

sys.path.insert(1, os.path.join(sys.path[0], ".."))

In [3]:
import yaml
from easydict import EasyDict
import pickle

from tqdm.auto import tqdm

from registration_metrics import (
    seed_everything,
    CARLA_dataset,
    prepare_data,
    sampling_pipeline,
)

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


Считываем данные:

In [4]:
%%time
data = CARLA_dataset(start_id=0, end_id=2422, dataset_coords_type="world")

CPU times: user 1min 29s, sys: 1min 6s, total: 2min 35s
Wall time: 51.1 s


# Run with `affinity="cosine"`

In [6]:
with open('../configs/mom_pipeline/cosine_affinity.yaml', 'r') as file:
    config = EasyDict(yaml.safe_load(file))

config

{'normals_estimation': {'max_radius': 1.0, 'max_nn': 30},
 'orth_planes_extraction': {'min_clust_size': 10,
  'affinity': 'cosine',
  'dist_threshold': 0.1,
  'from_pc': 'first',
  'downsample': {'use': False, 'voxel_size': 0.6, 'upsample': False}},
 'planes_segmentation': {'algo': 'svd_filter'},
 'metrics': {'mom': {'min_nn': 6, 'input': 'whole'},
  'mean_map_metric': {'min_nn': 6}}}

In [ ]:
seed_everything(123)

prepare_func = prepare_data

map_size = 10
N_samples = 5
cov_scaler = 10

results = {}
for internal_id in tqdm(range(0, 2422, 40), "Internal IDs"):
    start_id, end_id = internal_id, internal_id + map_size

    data = CARLA_dataset(start_id=start_id, end_id=end_id, dataset_coords_type="world")

    pcs = data.clds
    T_gt = data.poses

    pcs, T_gt, orth_planes, orth_planes_cld_ind, details = prepare_func(
        pcs, T_gt, config)
    details["orth_planes"] = orth_planes
    details["orth_planes_cld_ind"] = orth_planes_cld_ind

    tips = EasyDict(orth_list=orth_planes, config=config)

    save_dict = sampling_pipeline(
        pcs=pcs, T_gt=T_gt, map_tips=tips,
        N_samples=N_samples, cov_scaler=cov_scaler,
        orth_planes_cld_ind=orth_planes_cld_ind
    )

    results[internal_id, map_size] = {**details, **save_dict}

with open(f"CARLA-cosine-affinity-orth-planes.pkl", 'wb') as sfile:
    pickle.dump(results, sfile)

# Run with `affinity="euclidean"`

In [9]:
with open('../configs/mom_pipeline/euclidean_affinity.yaml', 'r') as file:
    config = EasyDict(yaml.safe_load(file))

config

{'normals_estimation': {'max_radius': 1.0, 'max_nn': 30},
 'orth_planes_extraction': {'min_clust_size': 10,
  'affinity': 'euclidean',
  'dist_threshold': 0.1,
  'from_pc': 'first',
  'downsample': {'use': False, 'voxel_size': 0.6, 'upsample': False}},
 'planes_segmentation': {'algo': 'svd_filter'},
 'metrics': {'mom': {'min_nn': 6, 'input': 'whole'},
  'mean_map_metric': {'min_nn': 6}}}

In [ ]:
seed_everything(123)

prepare_func = prepare_data

map_size = 1

results = {}
for external_id in tqdm(range(0, len(data), 40), "Internal IDs"):
    internal_id = external_id + data.start_id

    pcs = data.clds[external_id:external_id + map_size]
    T_gt = data.poses[external_id:external_id + map_size]

    pcs, T_gt, orth_planes, orth_planes_cld_ind, details = prepare_func(
        pcs, T_gt, config)

    details["orth_planes"] = orth_planes
    results[internal_id] = details
    
with open("CARLA-euclidean-affinity-orth-planes.pkl", 'wb') as sfile:
    pickle.dump(results, sfile)